In [1]:
# 拟合并且保存图片
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.patches import Circle
import os
# from scipy.optimize import curve_fit


def circle(x, cx, cy, r):
    return np.sqrt(r**2 - (x - cx)**2) + cy

# 创建一个列表，用于存储圆心和半径
circle_data = []
error_data = []
# 读取黑白图片
input_folder = './p2-7_extracted_cropped_binary/p4_extracted/'
output_folder = './fitted_extracted/fitted_p4/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
# os.path.join(output_path, 'fitted_'+filename)

i=1

for filename in os.listdir(input_folder):
    print(i)
    i=i+1

    # 创建一个新的 Figure 对象
    plt.figure()
    image = cv2.imread(os.path.join(input_folder, filename), cv2.IMREAD_GRAYSCALE)
    # image = cv2.imread('./cropped/cropped_p2-1_0015.bmp', cv2.IMREAD_GRAYSCALE)
    width, length = image.shape
    print(width)
    
    cropped_image = image

    # 获取裁剪后图像的形状
    rows, cols = cropped_image.shape

    # 存储每列的第一个黑色点的坐标
    first_black_pixels = []

    # 遍历每一列
    for col in range(cols):
    # 找到当前列中第一个黑色点的行坐标
        black_pixel_row = np.argmax(cropped_image[:, col] == 0)
        
        # 如果找到了黑色点，则记录坐标
        if black_pixel_row > 0:  # 0 表示未找到黑色点
            # 转换坐标为完整图像坐标
            full_image_row =width - (0 + black_pixel_row)
            first_black_pixels.append((0 + col, full_image_row))

    # 将坐标转换为 NumPy 数组
    first_black_pixels = np.array(first_black_pixels)
    # print(first_black_pixels)

    # 进行圆形拟合
    # p0 = [np.mean(first_black_pixels[:, 0]), np.mean(first_black_pixels[:, 1]), 10.0]  # 初始猜测值
    p0 = [np.mean(first_black_pixels[:, 0]), np.mean(first_black_pixels[:, 1])-200, 200.0]  # 初始猜测值
    params, covariance = curve_fit(circle, first_black_pixels[:, 0], first_black_pixels[:, 1], p0=p0)


    predicted_y = circle(first_black_pixels[:, 0], *params)

    # 计算拟合误差
    y_loss = np.sqrt(np.mean((first_black_pixels[:, 1] - predicted_y)**2))
    print('y_loss:', y_loss)
    r_squared = 1 - (np.sum((first_black_pixels[:, 1] - circle(first_black_pixels[:, 0], *params))**2) / np.sum((first_black_pixels[:, 1] - np.mean(first_black_pixels[:, 1]))**2))
    print('r_squared:', r_squared)


    error_data.append((round(y_loss, 3), round(r_squared, 3)))



#     fit_errors_pix = np.sqrt(np.diag(covariance))
#     # 提取拟合结果
#     cx, cy, r = params

#     # 生成拟合曲线的 x 值范围
#     fit_x = np.linspace(min(first_black_pixels[:, 0]), max(first_black_pixels[:, 0]), 100)
    
#     # 计算拟合曲线的 y 值
#     fit_y = circle(fit_x, cx, cy, r)

#     # 显示图像

#     plt.imshow(image, cmap='gray')

#     # 在坐标轴上画出每列的第一个黑色点的位置
#     plt.scatter(first_black_pixels[:, 0], width-first_black_pixels[:, 1], color='red', s=5)
#     # 画出拟合圆
#     # plt.plot(fit_x, fit_y, color='blue')
#     circle_patch = Circle((cx, width-cy), r, color='purple', fill=False)
#     plt.gca().add_patch(circle_patch)


#     scale_factor = 41 / length  # 将369像素映射到29mm

#     # 标记圆心位置
#     plt.scatter(cx, width-cy, color='green', marker='x')

#     p1= [1.2, 7.8]
#     p1_scaled=[0,0]
#     p1_scaled[0] = (p1[0]+13)/scale_factor
#     p1_scaled[1] = width - (p1[1]+6)/scale_factor 
#     print( "焊接位置/mm：", p1[0]+13, p1[1]+6) 

#     print(p1_scaled)
#     plt.scatter(p1_scaled[0], p1_scaled[1], color='red', marker='x')

#     # # 显示图像、黑色像素位置和拟合圆
#     # plt.axis('equal')
#     # plt.show()
#     output_filename = os.path.join(output_folder, 'fitted_'+filename.replace('bmp','jpg'))
#     plt.savefig(output_filename)
#     print("圆半径",r*scale_factor,"mm")


#     scaled_first_black_pixels = first_black_pixels * scale_factor
#     # p0_scaled = [175 * scale_factor, 60 * scale_factor, 150.0 * scale_factor]  # 初始猜测值
#     p0_scaled = [p * scale_factor for p in p0]
#     params_scaled, covariance_scaled = curve_fit(circle, scaled_first_black_pixels[:, 0], scaled_first_black_pixels[:, 1], p0=p0_scaled)
#     # 计算标准偏差作为拟合误差的估计
#     fit_errors_mm = np.sqrt(np.diag(covariance_scaled))
#     # print(f"Circle center (cx_scaled/mm, cy_scaled/mm): ({ cx*scale_factor -13:.2f} ± {fit_errors[0]:.2f}, {cy*scale_factor - 6:.2f} ± {fit_errors[1]:.2f})")
#     # print(f"Circle radius (r_scaled/mm): {r*scale_factor:.2f} ± {fit_errors[2]:.2f}")

#     print(f"Circle center (cx_scaled/pix, cy_scaled/pix): ({(cx -170):.2f} ± {fit_errors_pix[0]:.2f}, {(580 - cy):.2f} ± {fit_errors_pix[1]:.2f})")
#     print(f"Circle radius (r_scaled/pix): {r:.2f} ± {fit_errors_pix[2]:.2f}")
#     circle_data.append((round(cx , 3), round(580 - cy , 3), round(r, 3)))
#     error_data.append((round(fit_errors_pix[0], 3), round(fit_errors_pix[1], 3), round(fit_errors_pix[2], 3)))



# # 读取文件
# # with open('predict.txt', 'r') as f:
# with open('./Data/labels-240301-p1_p6.txt', 'r') as f:
#     reader = csv.reader(f)
#     data = list(reader)

# # 找到第一个第四、五、六列为空的行的索引
# start_index = next((i for i, row in enumerate(data) if all(x == '' for x in row[3:6])), len(data))

# # 添加新的数据
# # for i in range(start_index, start_index + len(circle_data[:200])):
# #     if i < len(data):
# #         data[i][3:6] = circle_data[i - start_index]
# #     else:
# #         data.append(circle_data[i - start_index] + [''] * (len(data[0]) - 6))

# for i in range (400, 600):
#     data[i][3:6] = circle_data[i - 400] 

# # 写回文件
# # with open('predict.txt', 'w', newline='') as f:
# with open('./Data/labels-240301-p1_p6.txt', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerows(data)


# cv2.imwrite(output_filename, image)

1
580
y_loss: 1.3654120669794074
r_squared: 0.904225320657604
2
580
y_loss: 1.294820646149352
r_squared: 0.9012485120466303
3
580
y_loss: 1.0672160085077111
r_squared: 0.925675351289094
4
580
y_loss: 1.036809819101753
r_squared: 0.9215673760960339
5
580
y_loss: 1.226541427226955
r_squared: 0.89702476165449
6
580
y_loss: 1.4698814427470406
r_squared: 0.8880325236420153
7
580
y_loss: 1.655725639571835
r_squared: 0.8764315573445207
8
580
y_loss: 1.480057684540105
r_squared: 0.8995470766384556
9
580
y_loss: 1.5361831404968445
r_squared: 0.8800400162499655
10
580
y_loss: 1.3395913089025542
r_squared: 0.925081055061437
11
580
y_loss: 1.2195520275631957
r_squared: 0.9409148698010472
12
580
y_loss: 1.1501053327466577
r_squared: 0.9541345930756922
13
580
y_loss: 1.0567783403070148
r_squared: 0.9605493244552111
14
580
y_loss: 1.043192027692761
r_squared: 0.964661981825305
15
580
y_loss: 1.1214322027289971
r_squared: 0.9632504465850872
16
580
y_loss: 1.1281407800494823
r_squared: 0.96595687548558

C:\Users\Kevin\AppData\Local\Temp\ipykernel_8760\2218890742.py:32: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()
C:\Users\Kevin\AppData\Local\Temp\ipykernel_8760\2218890742.py:13: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(r**2 - (x - cx)**2) + cy


580
y_loss: 1.2912880623518193
r_squared: 0.9775865181529992
38
580
y_loss: 1.2770690820955093
r_squared: 0.9772210243151072
39
580
y_loss: 1.4472089478386114
r_squared: 0.9712292882863488
40
580
y_loss: 1.3866770237146808
r_squared: 0.9742014440779994
41
580
y_loss: 1.5467878330980591
r_squared: 0.9734124199479931
42
580
y_loss: 1.4163655303070497
r_squared: 0.9751970092223778
43
580
y_loss: 1.55237475334648
r_squared: 0.9717696549416046
44
580
y_loss: 1.6340157659946288
r_squared: 0.9716441369956598
45
580
y_loss: 1.3294690550705028
r_squared: 0.9841191795744869
46
580
y_loss: 1.069458210025929
r_squared: 0.9898793041261893
47
580
y_loss: 1.1957505386348155
r_squared: 0.9874451062088154
48
580
y_loss: 1.2617090217549596
r_squared: 0.9868317507190185
49
580
y_loss: 1.1954026907803386
r_squared: 0.9896723278206502
50
580
y_loss: 1.2374156843389197
r_squared: 0.9884773489015296
51
580
y_loss: 1.3825012556489196
r_squared: 0.9868637565355677
52
580
y_loss: 1.3382953705409777
r_squared: 0

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [2]:
with open('error.txt', 'r') as f:
    reader = csv.reader(f)
    error_txt = list(reader)
print(len(error_txt))

# for _ in range(1200 - len(error_txt)):
#     error_txt.append([''] * 3)
# print(len(error_txt))

# # 添加新的数据
for i in range(len(error_txt)):
    # 如果原有的列数不足6列，用空字符串填充
    while len(error_txt[i]) < 5:
        error_txt[i].append('')

for i in range (400, 600):
    error_txt[i][0:2] = error_data[i - 400]
    # 写回文件
with open('error.txt', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(error_txt)

1200
